In [1]:
import pyximport; pyximport.install()
from utils import *
import time
import pandas as pd

In [2]:
n_instance = input()

 A1


In [3]:
folder_instances = 'Instances/'
folder_rpa_solutions = 'RPA_Solutions/'
path = folder_instances+n_instance+'.txt'
path_solutions =folder_rpa_solutions+'Detailed_Solution_'+n_instance+'.txt'

In [4]:
%%time
start = time.time()
count, number_of_customers, one, number_of_vehicles,\
vect_cord, vect_domanda, vect_carico, vec_b, vec_l, capacity = parser_instance(path)  

distance = distance_matrix(vect_cord)
saving=saving_matrix(distance)
dict_Saving_Order=savingOrder(saving)
dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)
list_vehicle, set_visited = init_list_of_vehicle(dict_l, vect_domanda, distance, number_of_vehicles)
route_linehaul(dict_l, list_vehicle, vect_domanda, set_visited, distance,  capacity) #funzione void
route_backhaul(list_vehicle, dict_merge, dict_b, set_visited, vect_carico, distance, number_of_customers, capacity) #funzione void
tot_cost=sum([x.cost for x in list_vehicle])
duration = time.time()-start

CPU times: user 4.57 ms, sys: 1.1 ms, total: 5.66 ms
Wall time: 5.34 ms


In [5]:
df_data = pd.DataFrame({'': [number_of_customers, capacity, number_of_vehicles,
                            len(set_visited)-1, tot_cost, extract_total_cost_BP(path_solutions),
                            round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2), duration]},
                       index=['Customers', 'Max Load', 'Routes Of the Solution', 'nodes visited',
                              'Total Cost', 'Total Cost BP', 'Error', 'Time for Instance'
                             ])

In [6]:
df_data

,
Customers,25.000000
Max Load,1550.000000
Routes Of the Solution,8.000000
nodes visited,25.000000
Total Cost,285271.679110
Total Cost BP,229886.000000
Error,24.090000
Time for Instance,0.005327


In [7]:
#support
count = 0
best_route = []
flag = False

with open(path_solutions) as file:
    for line in file:
        if "ROUTE" in line:
            flag = True
        if flag:
            count+=1
        if count == 7:
            best_route.append([int(x) for x in line.split()])
            flag=False
            count=0


In [8]:
best_route

[[0, 16, 14, 4, 0],
 [0, 15, 8, 2, 0],
 [0, 11, 0],
 [0, 9, 20, 17, 5, 0],
 [0, 13, 25, 12, 10, 3, 0],
 [0, 18, 7, 21, 0],
 [0, 24, 22, 1, 0],
 [0, 23, 19, 6, 0]]

In [9]:
my_route=[x.route for x in list_vehicle]
my_route

[[0, 10, 12, 16, 4, 3, 5, 0],
 [0, 19, 23, 18, 1, 2, 0],
 [0, 13, 25, 11, 0],
 [0, 20, 22, 0],
 [0, 14, 21, 0],
 [0, 9, 24, 17, 0],
 [0, 6, 7, 0],
 [0, 8, 15, 0]]

In [10]:
best_cost=0
for element in best_route:
    for i,v in enumerate(element):
        if i > 0:
            best_cost+=distance[element[i-1]][element[i]]

In [11]:
my_cost=0
for element in my_route:
    for i,v in enumerate(element):
        if i > 0:
            my_cost+=distance[element[i-1]][element[i]]

In [12]:
best_cost

229885.64539163865

In [13]:
my_cost

285271.6791095759

In [14]:
round((my_cost * 100) / best_cost -100 , 2)

24.09